<a href="https://colab.research.google.com/github/parthagarwal04/Fake-News-Predictitor/blob/main/fake_news_classifier_using_logistic_regression_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Link For the Dataset: https://www.kaggle.com/c/fake-news/data?select=train.csv

Importing all the Libraries

In [1]:
import re
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Loading Dataset

In [14]:
data= pd.read_csv('/content/train.csv.zip')

In [16]:
data.shape


(20800, 5)

downloding stopwords to remove them from data for pre processing of data

In [32]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

printing first 25 news of data set

In [21]:
data.head(25)

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.comWhy the Truth Might Get You ...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss15 Civilians Killed In Single U...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard PortnoyIranian woman jailed for fiction...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0,Daniel NussbaumJackie Mason: Hollywood Would L...
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,,Ever wonder how Britain’s most iconic pop pian...,1,Life: Life Of Luxury: Elton John’s 6 Favorite ...
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0,Alissa J. RubinBenoît Hamon Wins French Social...
8,8,Excerpts From a Draft Script for Donald Trump’...,,Donald J. Trump is scheduled to make a highly ...,0,Excerpts From a Draft Script for Donald Trump’...
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0,Megan Twohey and Scott ShaneA Back-Channel Pla...


# counting the number values in the dataset where there is no value in title


In [18]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

Now we will add empty string whereever the data is null


In [19]:
data = data.fillna('')

Merging the author field with the news title field in field called 'content'.

In [20]:
data['content'] = data['author']+''+data['title']

In [27]:
# checking if they were combined succesfully by printing first
print(data['content'])

0        Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1        Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2        Consortiumnews.comWhy the Truth Might Get You ...
3        Jessica Purkiss15 Civilians Killed In Single U...
4        Howard PortnoyIranian woman jailed for fiction...
                               ...                        
20795    Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796    Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat...
20797    Michael J. de la Merced and Rachel AbramsMacy’...
20798    Alex AnsaryNATO, Russia To Hold Parallel Exerc...
20799               David SwansonWhat Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [29]:
#separating the data and label for model training
X = data.drop(columns='label', axis=1)
Y = data['label']
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

We create stopwords to clean the text

In [42]:
port_stem = PorterStemmer()
def stemming(content):
    cleaned_content = re.sub('[^a-zA-Z]', ' ', content).lower()
    words = cleaned_content.split()
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in words if word not in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_words)
    return stemmed_content


stemming the data now

In [43]:
data['content'] = data['content'].apply(stemming)

checking if all words are stemmed

In [44]:
print(data['content'])

0        darrel lucush dem aid even see comey letter ja...
1        daniel j flynnflynn hillari clinton big woman ...
2                consortiumnew comwhi truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoyiranian woman jail fiction unpub...
                               ...                        
20795    jerom hudsonrapp trump poster child white supr...
20796    benjamin hoffmann f l playoff schedul matchup ...
20797    michael j de la merc rachel abramsmaci said re...
20798    alex ansarynato russia hold parallel exerci ba...
20799                        david swansonwhat keep f aliv
Name: content, Length: 20800, dtype: object


In [60]:
#separating the data and label
content = data['content'].values
label = data['label'].values
print(content)
print(label)

['darrel lucush dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynnflynn hillari clinton big woman campu breitbart'
 'consortiumnew comwhi truth might get fire' ...
 'michael j de la merc rachel abramsmaci said receiv takeov approach hudson bay new york time'
 'alex ansarynato russia hold parallel exerci balkan'
 'david swansonwhat keep f aliv']
[1 0 1 ... 0 1 1]


Now We will convert Text data to Numberical data

In [61]:
vectorizer = TfidfVectorizer()
vectorizer.fit(content)

content = vectorizer.transform(content)

In [62]:
# now printing 'content' matrix where each column represents a paticular word in the vocaboulary and each row represents a news article
print (content)

  (0, 26171)	0.28088379401596425
  (0, 22568)	0.2552336018069161
  (0, 14920)	0.43006226759639316
  (0, 14460)	0.29177259684200296
  (0, 12695)	0.24619727512767195
  (0, 7961)	0.23133661742488731
  (0, 6233)	0.2839932825877813
  (0, 5930)	0.35488202138141456
  (0, 4975)	0.2472595823572816
  (0, 4187)	0.3625320323150658
  (0, 572)	0.2694167078545385
  (1, 27744)	0.37204408227449065
  (1, 11235)	0.2434639969074374
  (1, 8704)	0.5300315426405939
  (1, 5877)	0.3206218375341294
  (1, 4739)	0.23523739119536424
  (1, 3838)	0.44601381123255807
  (1, 3260)	0.18800278047529806
  (1, 2602)	0.35911932371235855
  (2, 26066)	0.3665032495181434
  (2, 16256)	0.43295215406038445
  (2, 9384)	0.30743020569262086
  (2, 8499)	0.3411947414020896
  (2, 5204)	0.40440534260277944
  (2, 5089)	0.5511414848555652
  :	:
  (20797, 25609)	0.08234347981192172
  (20797, 25155)	0.3125002447493035
  (20797, 21934)	0.2494515865666979
  (20797, 20637)	0.2734270447609076
  (20797, 20353)	0.25042469546773877
  (20797, 17396

Now spliting data into training and testing where 30 percent data is used for testing and 70 percent for traaining

In [64]:
content_train, content_test, label_train, label_test = train_test_split(content, label, test_size=0.3, stratify=label, random_state=3)

Training the Model Using logistic Regession

In [65]:
model = LogisticRegression()

In [66]:
model.fit(content_train, label_train)

LogisticRegression()

In [67]:
#Checking Accuacy of our model
content_train_prediction = model.predict(content_train)
training_data_accuracy = accuracy_score(content_train_prediction, label_train)

In [68]:
print('The training data accuracy score is found to be : ', training_data_accuracy)

The training data accuracy score is found to be :  0.9699175824175824


In [69]:
predicted_labels = model.predict(content_test)
accuracy = accuracy_score(predicted_labels, label_test)

In [70]:
print('The testing data accuracy score is found to be : ', accuracy)

The testing data accuracy score is found to be :  0.9514423076923076


Now we can make program to predict the news weather it is fake or real using our model where the news article is entered by the user and stored in the variable new_data_point

In [ ]:
new_data_point = content_test[3]

prediction = model.predict([new_data_point])[0]

if prediction == 0:
    print('The news is most likely is Real')
else:
    print('The news is most likely Fake')